In [ ]:
import cv2
import pyautogui
from time import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose
 
# Setup the Pose function for images.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=1)
 
# Setup the Pose function for videos.
pose_video = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.7,min_tracking_confidence=0.7)
 
# Initialize mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils 



def detectPose(image, pose, draw=False, display=False):
    '''
    This function performs the pose detection on the most prominent person in an image.
    Args:
        image:   The input image with a prominent person whose pose landmarks needs to be detected.
        pose:    The pose function required to perform the pose detection.
        draw:    A boolean value that is if set to true the function draw pose landmarks on the output image. 
        display: A boolean value that is if set to true the function displays the original input image, and the 
                 resultant image and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn if it was specified.
        results:      The output of the pose landmarks detection on the input image.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Check if any landmarks are detected and are specified to be drawn.
    if results.pose_landmarks and draw:
    
        # Draw Pose Landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec = mp_drawing.DrawingSpec(color= (0, 0, 255), thickness=2, circle_radius=2),
                                connection_drawing_spec = mp_drawing.DrawingSpec(color=(0 ,128,0), thickness=2, circle_radius=2))
 
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121)
        plt.imshow(image[:,:,::-1])
        plt.title("Original Image")
        plt.axis('off')
        plt.subplot(122)
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    
    # Otherwise
    else:

        # Return the output image and the results of pose landmarks detection.
        return output_image, results
    
    
    
    
def checkHandsJoined(image, results, draw=False, display=False):
    '''
    This function checks whether the hands of the person are joined or not in an image.
    Args:
        image:   The input image with a prominent person whose hands status (joined or not) needs to be classified.
        results: The output of the pose landmarks detection on the input image.
        draw:    A boolean value that is if set to true the function writes the hands status & distance on the output image. 
        display: A boolean value that is if set to true the function displays the resultant image and returns nothing.
    Returns:
        output_image: The same input image but with the classified hands status written, if it was specified.
        hand_status:  The classified status of the hands whether they are joined or not.
    '''
    
    # Get the height and width of the input image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the hands status label on.
    output_image = image.copy()
    
    # Get the left wrist landmark x and y coordinates.
    left_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * width,
                          results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * height)

    # Get the right wrist landmark x and y coordinates.
    right_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                           results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * height)
    
    # Calculate the euclidean distance between the left and right wrist.
    euclidean_distance = int(hypot(left_wrist_landmark[0] - right_wrist_landmark[0],
                                   left_wrist_landmark[1] - right_wrist_landmark[1]))
    
    # Compare the distance between the wrists with a appropriate threshold to check if both hands are joined.
    if euclidean_distance < 130:
        
        # Set the hands status to joined.
        hand_status = 'Hands Joined'
        
        # Set the color value to green.
        color = (0, 255, 0)
        
    # Otherwise.    
    else:
        
        # Set the hands status to not joined.
        hand_status = 'Hands Not Joined'
        
        # Set the color value to red.
        color = (0, 0, 255)
        
    # Check if the Hands Joined status and hands distance are specified to be written on the output image.
    if draw:

        # Write the classified hands status on the image. 
        cv2.putText(output_image, hand_status, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        
        # Write the the distance between the wrists on the image. 
        cv2.putText(output_image, f'Distance: {euclidean_distance}', (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        
    # Check if the output image is specified to be displayed.
    if display:

        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    
    # Otherwise
    else:
    
        # Return the output image and the classified hands status indicating whether the hands are joined or not.
        return output_image, hand_status
    



def checkLeftRight(image, results, draw=False, display=False):
    '''
    This function finds the horizontal position (left, center, right) of the person in an image.
    Args:
        image:   The input image with a prominent person whose the horizontal position needs to be found.
        results: The output of the pose landmarks detection on the input image.
        draw:    A boolean value that is if set to true the function writes the horizontal position on the output image. 
        display: A boolean value that is if set to true the function displays the resultant image and returns nothing.
    Returns:
        output_image:         The same input image but with the horizontal position written, if it was specified.
        horizontal_position:  The horizontal position (left, center, right) of the person in the input image.
    '''
    
    # Declare a variable to store the horizontal position (left, center, right) of the person.
    horizontal_position = None
    
    # Get the height and width of the image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the horizontal position on.
    output_image = image.copy()
    
    # Retreive the x-coordinate of the left shoulder landmark.
    left_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width)
 
    # Retreive the x-corrdinate of the right shoulder landmark.
    right_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width)
    
    # Check if the person is at left that is when both shoulder landmarks x-corrdinates
    # are less than or equal to the x-corrdinate of the center of the image.
    if (right_x <= width//2 and left_x <= width//2):
        
        # Set the person's position to left.
        horizontal_position = 'Left'
 
    # Check if the person is at right that is when both shoulder landmarks x-corrdinates
    # are greater than or equal to the x-corrdinate of the center of the image.
    elif (right_x >= width//2 and left_x >= width//2):
        
        # Set the person's position to right.
        horizontal_position = 'Right'
    
    # Check if the person is at center that is when right shoulder landmark x-corrdinate is greater than or equal to
    # and left shoulder landmark x-corrdinate is less than or equal to the x-corrdinate of the center of the image.
    elif (right_x >= width//2 and left_x <= width//2):
        
        # Set the person's position to center.
        horizontal_position = 'Center'
        
    # Check if the person's horizontal position and a line at the center of the image is specified to be drawn.
    if draw:
 
        # Write the horizontal position of the person on the image. 
        cv2.putText(output_image, horizontal_position, (5, height - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        
        # Draw a line at the center of the image.
        cv2.line(output_image, (width//2, 0), (width//2, height), (255, 255, 255), 2)
        
    # Check if the output image is specified to be displayed.
    if display:
 
        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    
    # Otherwise
    else:
    
        # Return the output image and the person's horizontal position.
        return output_image, horizontal_position



def checkJumpCrouch(image, results, MID_Y=250, draw=False, display=False):
    '''
    This function checks the posture (Jumping, Crouching or Standing) of the person in an image.
    Args:
        image:   The input image with a prominent person whose the posture needs to be checked.
        results: The output of the pose landmarks detection on the input image.
        MID_Y:   The intial center y-coordinate of both shoulders landmarks of the person recorded during starting
                 the game. This will give the idea of the person's height when he is standing straight.
        draw:    A boolean value that is if set to true the function writes the posture on the output image. 
        display: A boolean value that is if set to true the function displays the resultant image and returns nothing.
    Returns:
        output_image: The input image with the person's posture written, if it was specified.
        posture:      The posture (Jumping, Crouching or Standing) of the person in an image.
    '''
    
    # Get the height and width of the image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the posture label on.
    output_image = image.copy()
    
    # Retreive the y-coordinate of the left shoulder landmark.
    left_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height)
 
    # Retreive the y-coordinate of the right shoulder landmark.
    right_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * height)
 
    # Calculate the y-coordinate of the mid-point of both shoulders.
    actual_mid_y = abs(right_y + left_y) // 2
    
    # Calculate the upper and lower bounds of the threshold.
    lower_bound = MID_Y-35
    upper_bound = MID_Y+35
    
    # Check if the person has jumped that is when the y-coordinate of the mid-point 
    # of both shoulders is less than the lower bound.
    if (actual_mid_y < lower_bound):
        
        # Set the posture to jumping.
        posture = 'Jumping'
    
    # Check if the person has crouched that is when the y-coordinate of the mid-point 
    # of both shoulders is greater than the upper bound.
    elif (actual_mid_y > upper_bound):
        
        # Set the posture to crouching.
        posture = 'Crouching'
    
    # Otherwise the person is standing and the y-coordinate of the mid-point 
    # of both shoulders is between the upper and lower bounds.    
    else:
        
        # Set the posture to Standing straight.
        posture = 'Standing'
        
    # Check if the posture and a horizontal line at the threshold is specified to be drawn.
    if draw:
 
        # Write the posture of the person on the image. 
        cv2.putText(output_image, posture, (5, height - 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        
        # Draw a line at the intial center y-coordinate of the person (threshold).
        cv2.line(output_image, (0, MID_Y),(width, MID_Y),(255, 255, 255), 2)
        
    # Check if the output image is specified to be displayed.
    if display:
 
        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    
    # Otherwise
    else:
    
        # Return the output image and posture indicating whether the person is standing straight or has jumped, or crouched.
        return output_image, posture
    


# Main loop moved into a function to avoid automatic webcam use during import/run

def main(num_of_frames=15, run_once=False):
    """Main loop for webcam processing. Set run_once=True to process a single frame (useful for testing)."""
    # Initialize the VideoCapture object to read from the webcam.
    camera_video = cv2.VideoCapture(0)
    camera_video.set(3,1280)
    camera_video.set(4,960)
    
    # Create named window for resizing purposes.
    cv2.namedWindow('Subway Surfers with Pose Detection', cv2.WINDOW_NORMAL)
    
    # Initialize a variable to store the time of the previous frame.
    time1 = 0
    
    # Initialize a variable to store the state of the game (started or not).
    game_started = False   
    
    # Initialize a variable to store the index of the current horizontal position of the person.
    # At Start the character is at center so the index is 1 and it can move left (value 0) and right (value 2).
    x_pos_index = 1
    
    # Initialize a variable to store the index of the current vertical posture of the person.
    # At Start the person is standing so the index is 1 and he can crouch (value 0) and jump (value 2).
    y_pos_index = 1
    
    # Declate a variable to store the intial y-coordinate of the mid-point of both shoulders of the person.
    MID_Y = None
    
    # Initialize a counter to store count of the number of consecutive frames with person's hands joined.
    counter = 0
    
    # Initialize the number of consecutive frames on which we want to check if person hands joined before starting the game.
    # Taken from function argument num_of_frames
    
    # Iterate until the webcam is accessed successfully.
    while camera_video.isOpened():
        
        # Read a frame.
        ok, frame = camera_video.read()
        
        # Check if frame is not read properly then continue to the next iteration to read the next frame.
        if not ok:
            continue
        
        # Flip the frame horizontally for natural (selfie-view) visualization.
        frame = cv2.flip(frame, 1)
        
        # Get the height and width of the frame of the webcam video.
        frame_height, frame_width, _ = frame.shape
        
        # Perform the pose detection on the frame.
        frame, results = detectPose(frame, pose_video, draw=game_started)
        
        # Check if the pose landmarks in the frame are detected.
        if results.pose_landmarks:
            
            # Check if the game has started
            if game_started:
                
                # Commands to control the horizontal movements of the character.
                #--------------------------------------------------------------------------------------------------------------
                
                # Get horizontal position of the person in the frame.
                frame, horizontal_position = checkLeftRight(frame, results, draw=True)
                
                # Check if the person has moved to left from center or to center from right.
                if (horizontal_position=='Left' and x_pos_index!=0) or (horizontal_position=='Center' and x_pos_index==2):
                    
                    # Press the left arrow key.
                    pyautogui.press('left')
                    
                    # Update the horizontal position index of the character.
                    x_pos_index -= 1               
 
                # Check if the person has moved to Right from center or to center from left.
                elif (horizontal_position=='Right' and x_pos_index!=2) or (horizontal_position=='Center' and x_pos_index==0):
                    
                    # Press the right arrow key.
                    pyautogui.press('right')
                    
                    # Update the horizontal position index of the character.
                    x_pos_index += 1
                
                #--------------------------------------------------------------------------------------------------------------
            
            # Otherwise if the game has not started    
            else:
                
                # Write the text representing the way to start the game on the frame. 
                cv2.putText(frame, 'JOIN BOTH HANDS TO START THE GAME.', (5, frame_height - 10), cv2.FONT_HERSHEY_PLAIN,
                            2, (0, 255, 0), 3)
            
            # Command to Start or resume the game.
            #------------------------------------------------------------------------------------------------------------------
            
            # Check if the left and right hands are joined.
            if checkHandsJoined(frame, results)[1] == 'Hands Joined':
 
                # Increment the count of consecutive frames with +ve condition.
                counter += 1
 
                # Check if the counter is equal to the required number of consecutive frames.  
                if counter == num_of_frames:
 
                    # Command to Start the game first time.
                    #----------------------------------------------------------------------------------------------------------
                    
                    # Check if the game has not started yet.
                    if not(game_started):
 
                        # Update the value of the variable that stores the game state.
                        game_started = True
 
                        # Retreive the y-coordinate of the left shoulder landmark.
                        left_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * frame_height)
 
                        # Retreive the y-coordinate of the right shoulder landmark.
                        right_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height)
 
                        # Calculate the intial y-coordinate of the mid-point of both shoulders of the person.
                        MID_Y = abs(right_y + left_y) // 2
 
                        # Move to 1300, 800, then click the left mouse button to start the game.
                        pyautogui.click(x=1300, y=800, button='left')
                    
                    #----------------------------------------------------------------------------------------------------------
 
                    # Command to resume the game after death of the character.
                    #----------------------------------------------------------------------------------------------------------
                    
                    # Otherwise if the game has started.
                    else:
 
                        # Press the space key.
                        pyautogui.press('space')
                    
                    #----------------------------------------------------------------------------------------------------------
                    
                    # Update the counter value to zero.
                    counter = 0
 
            # Otherwise if the left and right hands are not joined.        
            else:
 
                # Update the counter value to zero.
                counter = 0
                
            #------------------------------------------------------------------------------------------------------------------
 
            # Commands to control the vertical movements of the character.
            #------------------------------------------------------------------------------------------------------------------
            
            # Check if the intial y-coordinate of the mid-point of both shoulders of the person has a value.
            if MID_Y:
                
                # Get posture (jumping, crouching or standing) of the person in the frame. 
                frame, posture = checkJumpCrouch(frame, results, MID_Y, draw=True)
                
                # Check if the person has jumped.
                if posture == 'Jumping' and y_pos_index == 1:
 
                    # Press the up arrow key
                    pyautogui.press('up')
                    
                    # Update the veritcal position index of  the character.
                    y_pos_index += 1 
 
                # Check if the person has crouched.
                elif posture == 'Crouching' and y_pos_index == 1:
 
                    # Press the down arrow key
                    pyautogui.press('down')
                    
                    # Update the veritcal position index of the character.
                    y_pos_index -= 1
                
                # Check if the person has stood.
                elif posture == 'Standing' and y_pos_index   != 1:
                    
                    # Update the veritcal position index of the character.
                    y_pos_index = 1
            
            #------------------------------------------------------------------------------------------------------------------
        
        
        # Otherwise if the pose landmarks in the frame are not detected.       
        else:
 
            # Update the counter value to zero.
            counter = 0
            
        # Calculate the frames updates in one second
        #----------------------------------------------------------------------------------------------------------------------
        
        # Set the time for this frame to the current time.
        time2 = time()
        
        # Check if the difference between the previous and this frame time > 0 to avoid division by zero.
        if (time2 - time1) > 0:
        
            # Calculate the number of frames per second.
            frames_per_second = 1.0 / (time2 - time1)
            
            # Write the calculated number of frames per second on the frame. 
            cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
        
        # Update the previous frame time to this frame time.
        # As this frame will become previous frame in next iteration.
        time1 = time2
        
        #----------------------------------------------------------------------------------------------------------------------
        
        # Display the frame.            
        cv2.imshow('Subway Surfers with Pose Detection', frame)
        
        # Wait for 1ms. If a a key is pressed, retreive the ASCII code of the key.
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
        if run_once:
            # process only one frame for testing
            break

    # Release the VideoCapture Object and close the windows.                  
    camera_video.release()
    cv2.destroyAllWindows()


# Safety guard: do not start webcam automatically when importing/defining
if __name__ == '__main__':
    RUN_MAIN = False  # Set to True to run webcam loop
    if RUN_MAIN:
        main()
    else:
        print("RUN_MAIN is False — main loop not started. To run, set RUN_MAIN = True or call main() manually.")


In [ ]:
# Quick non-webcam test: run detectPose on a blank image to verify functions run without error
import numpy as np
img = np.zeros((480,640,3), dtype=np.uint8)
out_img, res = detectPose(img, pose_image, draw=False, display=False)
print('pose_landmarks present:', bool(res.pose_landmarks))
print('output image shape:', out_img.shape)

In [1]:
import cv2
import pyautogui
from time import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose

# Setup the Pose function for images.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=1)

# Setup the Pose function for videos.
pose_video = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils 

def detectPose(image, pose, draw=False, display=False):
    '''
    This function performs the pose detection on the most prominent person in an image.
    '''
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Check if any landmarks are detected and are specified to be drawn.
    if results.pose_landmarks and draw:
        # Draw Pose Landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2))

    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121)
        plt.imshow(image[:,:,::-1])
        plt.title("Original Image")
        plt.axis('off')
        plt.subplot(122)
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    else:
        return output_image, results

def checkHandsJoined(image, results, draw=False, display=False):
    '''
    This function checks whether the hands of the person are joined or not in an image.
    '''
    height, width, _ = image.shape
    output_image = image.copy()
    
    # Get the left wrist landmark x and y coordinates.
    left_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * width,
                          results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * height)

    # Get the right wrist landmark x and y coordinates.
    right_wrist_landmark = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * width,
                           results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * height)
    
    # Calculate the euclidean distance between the left and right wrist.
    euclidean_distance = int(hypot(left_wrist_landmark[0] - right_wrist_landmark[0],
                                   left_wrist_landmark[1] - right_wrist_landmark[1]))
    
    # Compare the distance between the wrists with a appropriate threshold.
    if euclidean_distance < 130:
        hand_status = 'Hands Joined'
        color = (0, 255, 0)
    else:
        hand_status = 'Hands Not Joined'
        color = (0, 0, 255)
        
    if draw:
        cv2.putText(output_image, hand_status, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        cv2.putText(output_image, f'Distance: {euclidean_distance}', (10, 70),
                    cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        
    if display:
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    else:
        return output_image, hand_status

def checkLeftRight(image, results, draw=False, display=False):
    '''
    This function finds the horizontal position (left, center, right) of the person in an image.
    '''
    horizontal_position = None
    height, width, _ = image.shape
    output_image = image.copy()
    
    # Retrieve the x-coordinate of the left shoulder landmark.
    left_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * width)

    # Retrieve the x-coordinate of the right shoulder landmark.
    right_x = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * width)
    
    # Logic for Left/Right/Center
    if (right_x <= width//2 and left_x <= width//2):
        horizontal_position = 'Left'
    elif (right_x >= width//2 and left_x >= width//2):
        horizontal_position = 'Right'
    elif (right_x >= width//2 and left_x <= width//2):
        horizontal_position = 'Center'
        
    if draw:
        cv2.putText(output_image, horizontal_position, (5, height - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.line(output_image, (width//2, 0), (width//2, height), (255, 255, 255), 2)
        
    if display:
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    else:
        return output_image, horizontal_position

def checkJumpCrouch(image, results, MID_Y=250, draw=False, display=False):
    '''
    This function checks the posture (Jumping, Crouching or Standing).
    '''
    height, width, _ = image.shape
    output_image = image.copy()
    
    # Retrieve the y-coordinate of the left/right shoulder landmark.
    left_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * height)
    right_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * height)

    # Calculate the y-coordinate of the mid-point of both shoulders.
    actual_mid_y = abs(right_y + left_y) // 2
    
    # Calculate the upper and lower bounds of the threshold.
    lower_bound = MID_Y - 35
    upper_bound = MID_Y + 35
    
    if (actual_mid_y < lower_bound):
        posture = 'Jumping'
    elif (actual_mid_y > upper_bound):
        posture = 'Crouching'
    else:
        posture = 'Standing'
        
    if draw:
        cv2.putText(output_image, posture, (5, height - 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        cv2.line(output_image, (0, MID_Y),(width, MID_Y),(255, 255, 255), 2)
        
    if display:
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1])
        plt.title("Output Image")
        plt.axis('off')
    else:
        return output_image, posture


# --- MAIN EXECUTION START ---

# Initialize the VideoCapture object.
camera_video = cv2.VideoCapture(0)
camera_video.set(3, 1280)
camera_video.set(4, 960)

cv2.namedWindow('Subway Surfers with Pose Detection', cv2.WINDOW_NORMAL)

# Initialize variables.
time1 = 0
game_started = False   
x_pos_index = 1
y_pos_index = 1
MID_Y = None
counter = 0
num_of_frames = 15 # CORRECTED from num_of_framesnum_of_frames

while camera_video.isOpened():
    ok, frame = camera_video.read()
    if not ok:
        continue
    
    # Flip the frame horizontally for natural visualization.
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    
    # Perform pose detection.
    frame, results = detectPose(frame, pose_video, draw=game_started)
    
    if results.pose_landmarks:
        if game_started:
            # --- GAME CONTROL LOGIC ---
            
            # Horizontal Movements
            frame, horizontal_position = checkLeftRight(frame, results, draw=True)
            
            if (horizontal_position == 'Left' and x_pos_index != 0) or (horizontal_position == 'Center' and x_pos_index == 2):
                pyautogui.press('left')
                x_pos_index -= 1              

            elif (horizontal_position == 'Right' and x_pos_index != 2) or (horizontal_position == 'Center' and x_pos_index == 0):
                pyautogui.press('right')
                x_pos_index += 1
            
            # Vertical Movements
            if MID_Y:
                frame, posture = checkJumpCrouch(frame, results, MID_Y, draw=True)
                
                if posture == 'Jumping' and y_pos_index == 1:
                    pyautogui.press('up')
                    y_pos_index += 1 

                elif posture == 'Crouching' and y_pos_index == 1:
                    pyautogui.press('down')
                    y_pos_index -= 1
                
                elif posture == 'Standing' and y_pos_index != 1:
                    y_pos_index = 1
        
        else:
            # --- GAME START LOGIC ---
            cv2.putText(frame, 'JOIN BOTH HANDS TO START.', (5, frame_height - 10), 
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
        
            if checkHandsJoined(frame, results)[1] == 'Hands Joined':
                counter += 1
                
                # CORRECTED VARIABLE HERE
                if counter == num_of_frames: 
                    if not game_started:
                        game_started = True
                        
                        # Calibrate MID_Y based on current position
                        left_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * frame_height)
                        right_y = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height)
                        MID_Y = abs(right_y + left_y) // 2

                        # NOTE: You might need to adjust these coordinates based on your screen resolution!
                        # Alternatively, use pyautogui.press('space') if the game allows starting via keyboard.
                        # pyautogui.click(x=1300, y=800, button='left') 
                        pyautogui.press('space') # Changed to Space for safety
                    else:
                        pyautogui.press('space')
                    
                    counter = 0
            else:
                counter = 0

    else:
        counter = 0
        
    # FPS Calculation
    time2 = time()
    if (time2 - time1) > 0:
        frames_per_second = 1.0 / (time2 - time1)
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    time1 = time2
    
    cv2.imshow('Subway Surfers with Pose Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == 27: # Press ESC to exit
        break

camera_video.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import pyautogui
from time import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt

# --- INITIALIZATION ---

# Initialize mediapipe Hands class.
mp_hands = mp.solutions.hands

# Setup the Hands function for video.
# max_num_hands=2 allows us to detect both hands for the "Join Hands" start gesture.
hands_video = mp_hands.Hands(static_image_mode=False, 
                             max_num_hands=2, 
                             model_complexity=1, 
                             min_detection_confidence=0.7, 
                             min_tracking_confidence=0.7)

# Initialize mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils 

def detectHands(image, hands_model, draw=False):
    '''
    This function performs hand detection on the input image.
    '''
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Hand Detection.
    results = hands_model.process(imageRGB)
    
    # Check if landmarks are detected and are specified to be drawn.
    if results.multi_hand_landmarks and draw:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image=output_image, 
                landmark_list=hand_landmarks,
                connections=mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2)
            )
    
    return output_image, results

def checkHandsJoined(image, results, draw=False):
    '''
    Checks if two hands are close together (Joined) to start the game.
    '''
    height, width, _ = image.shape
    output_image = image.copy()
    hand_status = 'Hands Not Joined'
    color = (0, 0, 255)
    euclidean_distance = 0

    # We need exactly 2 hands to check if they are joined.
    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) == 2:
        
        # Get the wrist landmark of the first hand.
        wrist1 = (results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].x * width,
                  results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST].y * height)
        
        # Get the wrist landmark of the second hand.
        wrist2 = (results.multi_hand_landmarks[1].landmark[mp_hands.HandLandmark.WRIST].x * width,
                  results.multi_hand_landmarks[1].landmark[mp_hands.HandLandmark.WRIST].y * height)
        
        # Calculate distance.
        euclidean_distance = int(hypot(wrist1[0] - wrist2[0], wrist1[1] - wrist2[1]))
        
        # Threshold for joined hands.
        if euclidean_distance < 150:
            hand_status = 'Hands Joined'
            color = (0, 255, 0)

    if draw:
        cv2.putText(output_image, hand_status, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        if euclidean_distance > 0:
            cv2.putText(output_image, f'Distance: {euclidean_distance}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
            
    return output_image, hand_status

def getHandCommand(image, results, draw=False):
    '''
    Determines the command (Left, Right, Up, Down, Center) based on Index Finger position.
    Acts as a virtual joystick.
    '''
    command = 'Center'
    height, width, _ = image.shape
    output_image = image.copy()
    
    # Define the "Neutral Zone" (The box in the center).
    # Adjust these percentages to make the center box larger or smaller.
    center_x_min = width * 0.35
    center_x_max = width * 0.65
    center_y_min = height * 0.35
    center_y_max = height * 0.65

    if results.multi_hand_landmarks:
        # We only use the first detected hand for control to avoid confusion.
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # Track the INDEX_FINGER_TIP (Point 8).
        x = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * width
        y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * height

        # Determine position relative to the Neutral Zone.
        if x < center_x_min:
            command = 'Left'
        elif x > center_x_max:
            command = 'Right'
        elif y < center_y_min:
            command = 'Up'   # Jump
        elif y > center_y_max:
            command = 'Down' # Slide
        else:
            command = 'Center'

        if draw:
            # Draw the user's finger position
            cv2.circle(output_image, (int(x), int(y)), 10, (255, 255, 0), -1)
            
            # Draw the text
            cv2.putText(output_image, command, (5, height - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)

    if draw:
        # Draw the Neutral Zone Grid for visual feedback
        cv2.rectangle(output_image, (int(center_x_min), int(center_y_min)), 
                      (int(center_x_max), int(center_y_max)), (255, 255, 255), 2)

    return output_image, command

# --- MAIN EXECUTION START ---

# Initialize the VideoCapture object.
camera_video = cv2.VideoCapture(0)
camera_video.set(3, 1280)
camera_video.set(4, 960)

cv2.namedWindow('Subway Surfers - Hand Gesture Control', cv2.WINDOW_NORMAL)

# Initialize variables.
time1 = 0
game_started = False   
command_state = 'Center' # Tracks the current state to prevent spamming keys
counter = 0
num_of_frames = 15 

while camera_video.isOpened():
    ok, frame = camera_video.read()
    if not ok:
        continue
    
    # Flip the frame horizontally for natural interaction (Mirror view).
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    
    # Perform hand detection.
    frame, results = detectHands(frame, hands_video, draw=True)
    
    if results.multi_hand_landmarks:
        
        if game_started:
            # --- GAME CONTROL LOGIC (Virtual Joystick) ---
            
            frame, current_command = getHandCommand(frame, results, draw=True)
            
            # Only press the key if the command has CHANGED from the previous frame
            # or if we return to center, we reset the ability to press again.
            # This logic allows for "Press and release" behavior.
            
            if current_command != command_state:
                
                if current_command == 'Left':
                    pyautogui.press('left')
                elif current_command == 'Right':
                    pyautogui.press('right')
                elif current_command == 'Up':
                    pyautogui.press('up')
                elif current_command == 'Down':
                    pyautogui.press('down')
                
                # Update the state
                command_state = current_command
            
        else:
            # --- GAME START LOGIC ---
            cv2.putText(frame, 'JOIN HANDS TO START', (5, frame_height - 10), 
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
        
            frame, status = checkHandsJoined(frame, results, draw=True)
            
            if status == 'Hands Joined':
                counter += 1
                if counter == num_of_frames: 
                    if not game_started:
                        game_started = True
                        pyautogui.press('space') 
                        print("Game Started!")
                    counter = 0
            else:
                counter = 0

    else:
        # If hands are lost, reset counters
        counter = 0
        
    # FPS Calculation
    time2 = time()
    if (time2 - time1) > 0:
        frames_per_second = 1.0 / (time2 - time1)
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    time1 = time2
    
    cv2.imshow('Subway Surfers - Hand Gesture Control', frame)
    
    if cv2.waitKey(1) & 0xFF == 27: # Press ESC to exit
        break

camera_video.release()
cv2.destroyAllWindows()